In [10]:
from IPython.display import display
import IPython
import pandas as pd
from pandas import Series
import numpy as np
from random import randrange
from matplotlib import pyplot
import matplotlib.pyplot as plt
import sys
sys.path.append('..')
import math
from typing import Any, List, Dict, AnyStr, Optional
from pathlib import Path
from glob import glob
from utils.dataset import MarketData, DataPreprocessor, Plotting
from utils.windowgenerator import WindowGenerator, compile_and_fit
import tensorflow as tf
import talib
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [11]:
eua_options_cols = ['Date', 'Aggregate Put Open Interest  (R1)', 'Aggregate Call Open Interest  (R1)', 
                    'Aggregate Open Interest  (L1)', 'OPTION OI%', 'PUT/CALL OI']
eua_options = pd.read_excel('../data/data_sheet_latest.xlsx', sheet_name='EUA option-G363')

In [12]:
display(eua_options.head()[eua_options_cols])

,Date,Aggregate Put Open Interest (R1),Aggregate Call Open Interest (R1),Aggregate Open Interest (L1),OPTION OI%,PUT/CALL OI
0,2024-10-28 00:00:00,NaN,NaN,NaN,NaN,NaN
1,2024-10-24 00:00:00,203925.0,319362.0,790502.0,0.398304,1.566076
2,2024-10-23 00:00:00,203925.0,310977.0,790869.0,0.394328,1.524958
3,2024-10-22 00:00:00,203408.0,310342.0,792493.0,0.393304,1.525712
4,2024-10-21 00:00:00,202608.0,310892.0,790664.0,0.393739,1.534451


In [13]:
pachis_delta = pd.read_excel('../data/data_sheet_latest.xlsx', sheet_name='25Delta')
dec_cols = ["Date", "Hist Vol", "50D-Hist Vol", "50D", "25D Spread", "butterfly"]
pachis_delta_dec = pachis_delta[dec_cols]
pachis_delta_dec.columns = ["Date", "Hist Vol - 1Y", 
                        "iVol/Hist Vol Spread - Dec", 
                        "50 Delta iVol - Dec", 
                        "25Δ Risk Reversal (Call - Put) - Dec", 
                        "Butterfly - Dec"]

prompt_cols = ["Date.1", "50D.1", "25D Spread.1", "butterfly.1"]
pachis_delta_prompt = pachis_delta[prompt_cols]
pachis_delta_prompt.columns = ["Date", "50 Delta iVol - Prompt",
                               "25Δ Risk Reversal (Call - Put) - Prompt",
                               "Butterfly - Prompt"]

In [15]:
pachis_delta_dec

,Date,Hist Vol - 1Y,iVol/Hist Vol Spread - Dec,50 Delta iVol - Dec,25Δ Risk Reversal (Call - Put) - Dec,Butterfly - Dec
0,2024-10-25 00:00:00,28.34,8.534,36.874,-2.563,1.0375
1,2024-10-24 00:00:00,28.838,7.694,36.532,-2.277,0.9515
2,2024-10-23 00:00:00,28.275,8.343,36.618,-2.524,0.9790
3,2024-10-22 00:00:00,27.485,9.29,36.775,-3.375,1.1825
4,2024-10-21 00:00:00,27.491,9.286,36.777,-3.177,1.3265
...,...,...,...,...,...,...
470,NaN,NaN,NaN,NaN,NaN,NaN
471,NaN,NaN,NaN,NaN,NaN,NaN
472,NaN,NaN,NaN,NaN,NaN,NaN
473,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
option_ts = pd.read_excel('../data/data_sheet_latest.xlsx', sheet_name='Option Time series')

In [19]:
option_ts

,Date,Call_OI-Prompt,Put_OI-Prompt,Dec_OI,Unnamed: 4,Call/Put,Option%,Unnamed: 7,Date.1,Call OI,...,3M Trend.1,6M Trend.1,Unnamed: 24,Prompt Month,1M Trend.2,3M Trend.2,6M Trend.2,1M Trend.3,3M Trend.3,6M Trend.3
0,2024-10-25,142771,111895,389085,NaN,1.275937,0.395597,NaN,2024-10-25,142771,...,0.129551,0.179406,NaN,NaN,0.108416,0.120029,0.107733,-0.164985,-0.137882,0.138943
1,2024-10-24,134386,111895,389085,NaN,1.201001,0.387621,NaN,2024-10-24,134386,...,0.124274,0.175517,NaN,NaN,0.112584,0.121325,0.107167,-0.182038,-0.145167,0.136774
2,2024-10-23,133851,111895,389968,NaN,1.196220,0.386567,NaN,2024-10-23,133851,...,0.120846,0.173818,NaN,NaN,0.117018,0.122999,0.106909,-0.197935,-0.149422,0.136741
3,2024-10-22,133751,110578,391398,NaN,1.209562,0.384330,NaN,2024-10-22,133751,...,0.115659,0.170570,NaN,NaN,0.110476,0.117338,0.100194,-0.181173,-0.129316,0.155918
4,2024-10-21,132501,110503,389463,NaN,1.199072,0.384216,NaN,2024-10-21,132501,...,0.110779,0.167482,NaN,NaN,0.104077,0.111800,0.093632,-0.167295,-0.110091,0.174075
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
470,2023-01-06,14877,25887,261327,NaN,NaN,NaN,NaN,2023-01-06,29017,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
471,2023-01-05,14427,20937,262025,NaN,NaN,NaN,NaN,2023-01-05,28767,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
472,2023-01-04,13442,20412,278833,NaN,NaN,NaN,NaN,2023-01-04,28167,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
473,2023-01-03,12867,20912,280176,NaN,NaN,NaN,NaN,2023-01-03,28167,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
